In [ ]:
import os
import re
from pathlib import Path
from PIL import Image

# 設定
source_image_path = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Photoshop_annotation_data/Trap_door_muscle_fracture'
output_base_path = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation'

# 処理したい患者IDを直接指定
target_patient_id = "6870292"  # ここに患者IDを指定

def extract_patient_id(filename):
    """
    ファイル名から患者IDを抽出する関数
    例: 6375522_20_R_FLOOR-3.jpg から 6375522 を抽出
    """
    match = re.match(r'(\d+)_\d+', filename)
    if match:
        return match.group(1)
    return None

# 出力先ディレクトリの作成
output_dir = os.path.join(output_base_path, 'imagesTs')
os.makedirs(output_dir, exist_ok=True)

# すべての画像ファイルを取得
all_image_files = [f for f in os.listdir(source_image_path) if f.lower().endswith('.jpg')]

# 進捗表示用
total_files = len(all_image_files)
processed_count = 0
converted_count = 0

print(f"画像ファイル数: {total_files}")
print(f"対象患者ID: {target_patient_id}")

# 各ファイルを処理
for img_file in all_image_files:
    processed_count += 1
    
    # 患者IDを抽出
    patient_id = extract_patient_id(img_file)
    
    # 指定された患者IDのファイルのみを処理
    if patient_id == target_patient_id:
        try:
            # 画像を読み込みグレースケールに変換
            img_path = os.path.join(source_image_path, img_file)
            img = Image.open(img_path).convert('L')
            
            # ファイル名のベース部分を取得
            base_name = os.path.splitext(img_file)[0]
            
            # nnU-Net形式のファイル名で保存
            target_img_path = os.path.join(output_dir, f"{base_name}_0000.png")
            img.save(target_img_path, format='PNG')
            
            converted_count += 1
            print(f"変換: {img_file} -> {os.path.basename(target_img_path)}")
            
        except Exception as e:
            print(f"エラー: {img_file} の処理中にエラーが発生しました: {str(e)}")
    
    # 進捗表示
    if processed_count % 10 == 0 or processed_count == total_files:
        print(f"進捗: {processed_count}/{total_files} ファイル処理済み")

print(f"\n処理完了:")
print(f"処理されたファイル数: {processed_count}")
print(f"変換されたファイル数: {converted_count}")
print(f"出力先: {output_dir}")